# Functions

In [16]:
from matplotlib.pylab import imshow, figure, show, savefig, title
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import math
import os
from skimage import io
from skimage import color
import scipy.ndimage.filters as filters
    
#function to create a Gaussian kernel of shape (n,n) and sigma
def GaussKernel(shape=(5,5),sigma=1):
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

# function to apply gaussian filter to an image
def GaussianBlur(img, sig):
    win = GaussKernel((5,5),sig)
    blurred = filters.convolve(img, win, mode="constant", cval=0.0)
    return blurred

# function upsamples an image, smoothes it and then multiplies the results by a scalar of 4 to compensate for downsampling
def UpSample(img, previous, sig):
    #upsampling by adding all even rows and columns with zeros
    img = img.astype(np.float64)
    h = len(previous)
    w = len(previous[0])
    
    #account for the fact that images with odd/even shapes were downsampled differently (image.shape/2)
    if h%2 == 0: 
        for i in range(1, len(img)*2, 2):
            img = np.insert(img, i, 0, axis=0)
    if w%2 == 0:
        for j in range(1, len(img[0])*2, 2):
            img = np.insert(img, j, 0, axis=1)
    if h%2 != 0:
        for i in range(1, len(img)*2-2, 2):
            img = np.insert(img, i, 0, axis=0)
    if w%2 != 0:    
        for j in range(1, (len(img[0])*2)-2, 2):
            img = np.insert(img, j, 0, axis=1)
            
    #blurring image with same filter as downsampling
    img = GaussianBlur(img, sig)*4
    
    return img

# function creates and saves Laplacian and Gaussian Pyramids of an image
def CreatePyramids(img, filename, pyr_number, sig):
    pyrGauss = []
    pyrLap = []
    pyrGauss.append(img)

    #creating Gaussian Pyramid
    for i in range(1,pyr_number):
        blurred = GaussianBlur(pyrGauss[i-1], sig)
        downSample = blurred[::2, ::2].astype(np.float64) #downsample by deleting all even rows and columns
        pyrGauss.append(downSample)
    
    #creating Laplaccian Pyramid
    for n in range(pyr_number-1):
        current = pyrGauss[n+1]
        previous = pyrGauss[n]
        upSampled = UpSample(current, previous, sig)    
        pyrLap.append(pyrGauss[n] - upSampled) #Lap_n = Gauss_n - Gauss_n+1
    pyrLap.append(pyrGauss[pyr_number-1])
    
    #saving the Pyramids as images in folder Q1
    for i in range(len(pyrGauss)):
        figure(i)
        imshow(pyrGauss[i], cmap=cm.Greys_r)
        title(filename + "_G_" + str(i))
        savefig('Q1/' + filename + '_G_' + str(i) + '.png')
        
    for i in range(len(pyrLap)):
        figure(i)
        imshow(pyrLap[i], cmap=cm.Greys_r)
        title(filename + "_L_" + str(i))
        savefig('Q1/' + filename + '_L_' + str(i) + '.png')
        
    return pyrGauss, pyrLap

# function reconstructs an image from its Laplacian Pyramids
def Reconstruct(pyrLap, filename, sig):
    
    pyr_number = len(pyrLap)
    
    img = pyrLap[pyr_number-1]
    for n in range(pyr_number-2,-1,-1):
        previous = pyrLap[n]
        img = UpSample(img, previous, sig)
        img += previous
    
    img = ((img + 255)/2).astype(np.float64)
    
    figure(1)
    imshow(img, cmap=cm.Greys_r)
    title(filename + "_Reconstructed")
    savefig('Q1/' + filename + '_reconstructed_img.png')
    return img

# function to sharpen an image, given 2 images' Laplacian Pyramids
def EnhanceImage(pyrLap1, pyrLap2, sig):
    newPyrLap = []
    
    for n in range(0,len(pyrLap1)-1):
        mask = np.zeros(pyrLap1[n].shape)
        for i in range(len(pyrLap1[n])):
            for j in range(len(pyrLap1[n][i])):
                if abs(pyrLap1[n][i][j]) > abs(pyrLap2[n][i][j]):
                    mask[i][j] = pyrLap1[n][i][j]
                else:
                    mask[i][j] = pyrLap2[n][i][j]
        newPyrLap.append(mask)
    
    mask = np.zeros(pyrLap1[3].shape)
    for i in range(len(pyrLap1[3])):
        for j in range(len(pyrLap1[3][i])):
            mask[i][j] = (pyrLap1[3][i][j] + pyrLap2[3][i][j])/2
    newPyrLap.append(mask)
    
            
    recImg = Reconstruct(newPyrLap, "enhanced", sig)

# function compares two images to see if they are identical in both shape and pixel values
def CompareImages(ori_img, rec_img):
    array_diff = np.amax(ori_img-rec_img)
    if ori_img.shape == rec_img.shape and (array_diff <= 1 or array_diff >= -1):
        print "Identical"
    else:
        print "Not Identical"

    



# Task 2 - Building Pyramids

In [ ]:
#Build Gaussian and Laplacian Pyramids for the images mandril.tif and toucan.tif. What are the differences between the
#Laplacian Pyramids of the two images? Explain.

if __name__ == '__main__':
    if not os.path.exists("Q1"):
        os.makedirs("Q1")

    images = ['Images/Q1/mandril.tif',
              'Images/Q1/toucan.tif']
              
    for image in images:
        fn = image.split('/')[-1]
        img = io.imread(image)[0].astype(np.float64)
        pyrGauss, pyrLap = CreatePyramids(img, fn,5,1)



# Task 3 - Reconstructing Image

In [12]:
#Reconstruct the original images from their Laplacian Pyramids. Show in a simple way that the original images and 
#the reconstructed ones are identical.

if __name__ == '__main__':
    if not os.path.exists("Q1"):
        os.makedirs("Q1")

    images = ['Images/Q1/mandril.tif',
              'Images/Q1/toucan.tif']

    for image in images:
        fn = image.split('/')[-1]
        img = io.imread(image)[0].astype(np.float64)
        pyrGauss, pyrLap = CreatePyramids(img, fn, 5, 1)
        recImg = Reconstruct(pyrLap, fn, 1)
        CompareImages(img, recImg)
        

Identical
Identical


# Task 4 - Image Enhancement

In [17]:
#Implement an image enhancement method for the images focus1.tif and focus2.tif. The method should create a 
#single entirely-focused image from the two partially-focused ones. Use a pyramid with four levels. 
#Show the results and discuss their quality.

#reference http://www.cs.toronto.edu/~jepson/csc320/notes/pyramids.pdf

if __name__ == '__main__':
    if not os.path.exists("Q1"):
        os.makedirs("Q1")
    
    focus1 = io.imread('Images/Q1/focus1.tif').astype(np.float64)
    focus2 = io.imread('Images/Q1/focus2.tif').astype(np.float64)
    
    #create pyramids for both images
    pyrGauss1, pyrLap1 = CreatePyramids(focus1, 'focus1', 4, 1)
    pyrGauss2, pyrLap2 = CreatePyramids(focus2, 'focus2', 4, 1)
    
    EnhanceImage(pyrLap1, pyrLap2, 1)
    